In [120]:
from ultralytics import YOLO
import easyocr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [121]:
model  =  YOLO("runs/detect/train/weights/best.pt")

In [124]:
def draw_rects(image,coordinates=[],color=(0, 255, 0),text_tuple=None):
    
    for rects in coordinates:
        x,y,w,h =  rects
        rect_image =cv2.rectangle(image,(x,y),(w,h),color, 2)
        if text_tuple :
            rect_image = cv2.putText(text_tuple)
    
    return rect_image


def track_images(image_path, model):
    coordinates = []
    rect_images =  []
    # Run prediction and convert results to DataFrame
    results = model.predict(image_path)[0].boxes.data.detach().cpu().numpy()
    df = pd.DataFrame(results,columns=["x1","y1","x2","y2","conf","class_id"]).astype("float")
    
    # Iterate through each detection
    for _, row in df.iterrows():
        x1, y1, x2, y2, conf,_ = row  # it has only one class thus not needed
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        coordinates.append([x1, y1, x2, y2])
        rect_image = cv2.imread(image_path)[y1:y2, x1:x2]
        rect_images.append(rect_image)
    
    return (df,coordinates,conf,rect_images)




def draw_bounding_boxes(image, detections, threshold=0.25):
    for bbox, text, score in detections:
        if score > threshold:
            cv2.rectangle(image, tuple(map(int, bbox[0])), tuple(map(int, bbox[2])), (0, 255, 0), 5)
            cv2.putText(image, text, tuple(map(int, bbox[0])), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.65, (255, 0, 0), 2)

In [148]:
df,coordinates,conf,img =  track_images("C:/Users/RISHABH/Pictures/images/plate2.jpg",model)


image 1/1 C:\Users\RISHABH\Pictures\images\plate2.jpg: 448x640 1 clone-license-plate, 140.2ms
Speed: 2.5ms preprocess, 140.2ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)


In [149]:
img[0]

array([[[104, 114, 121],
        [104, 114, 121],
        [107, 116, 120],
        ...,
        [ 37,  49,  51],
        [ 52,  64,  66],
        [ 64,  76,  78]],

       [[ 70,  80,  87],
        [ 65,  75,  82],
        [ 68,  77,  81],
        ...,
        [ 33,  45,  47],
        [ 51,  63,  65],
        [ 66,  78,  80]],

       [[ 43,  53,  60],
        [ 29,  39,  46],
        [ 30,  39,  43],
        ...,
        [ 28,  40,  42],
        [ 49,  61,  63],
        [ 67,  79,  81]],

       ...,

       [[100, 104, 105],
        [ 63,  65,  66],
        [ 27,  29,  30],
        ...,
        [ 30,  31,  27],
        [ 64,  65,  61],
        [ 97,  98,  94]],

       [[102, 104, 104],
        [ 65,  67,  67],
        [ 28,  30,  30],
        ...,
        [ 34,  35,  31],
        [ 67,  68,  64],
        [101, 102,  98]],

       [[101, 105, 106],
        [ 65,  67,  68],
        [ 29,  31,  32],
        ...,
        [ 35,  38,  36],
        [ 68,  72,  67],
        [101, 104, 102]]

In [ ]:
img_ = np.array(Image.fromarray(img[0]))
reader = easyocr.Reader(['en'], gpu=False)
text_detections = reader.readtext(img_)
final_text = ""
for bbox, text, confidence in text_detections:
    final_text += text + " "
    print(f"Detected: '{text}' with confidence: {confidence:.2f}")

Using CPU. Note: This module is much faster with a GPU.
c:\codes\Lab\Number_plate_recognitio\plate\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Detected: 'N' with confidence: 0.93
Detected: '1075' with confidence: 0.65
Detected: 'CMI' with confidence: 0.38
Detected: '06' with confidence: 0.62
Detected: '21' with confidence: 0.17


In [153]:
print(len(text_detections))

5


In [151]:
Image.fromarray(img[0]).show()

In [152]:
final_text

'N 1075 CMI 06 21 '